In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import scipy
from scipy import stats
from matplotlib import pyplot as plt
# loading original module
import NeuroProcessing

In [5]:
#abr_lfpのz_scoreを収集する
deaf_files_path = Path("./deaf_datas")
deaf_files = list(deaf_files_path.glob("*"))
intact_files_path = Path("./intact_datas/")
intact_files = list(intact_files_path.glob("*"))
fault_files_path = Path("./fault_datas/")
fault_files = list(fault_files_path.glob("*"))

In [6]:
#pdのファイルの結合
deaf_datas = pd.concat([pd.read_csv(deaf_file) for deaf_file in deaf_files])
intact_datas = pd.concat([pd.read_csv(intact_file) for intact_file in intact_files])
deaf_datas = deaf_datas.dropna()
intact_datas = intact_datas.dropna()
fault_datas = pd.concat([pd.read_csv(fault_file) for fault_file in fault_files])
fault_datas = fault_datas.dropna()
deaf_sham = deaf_datas[deaf_datas["stim_params"] == "sham"]
intact_sham = intact_datas[intact_datas["stim_params"] == "sham"]
fault_sham = fault_datas[fault_datas["stim_params"] == "sham"]
all_sham_datas = pd.concat([deaf_sham,intact_sham,fault_sham])


In [7]:
all_sham_datas

,date,filename,stim_params,abr_z_score,abr_latency,lfp_z_score_100,lfp_latency_100,lfp_ampliture_100,lfp_z_score_200,lfp_latency_200,...,lfp_ampliture_500,lfp_z_score_600,lfp_latency_600,lfp_ampliture_600,lfp_z_score_700,lfp_latency_700,lfp_ampliture_700,lfp_z_score_800,lfp_latency_800,lfp_ampliture_800
16,20221012,03_us_cont_test,sham,3.194684,2.12,1.172409,31.0,-10.139079,1.806094,35.0,...,-13.087529,2.752312,26.0,-13.702036,2.735144,27.0,-11.316440,2.481509,29.0,-10.847396
11,20221118,03_us_cont_test2,sham,1.713978,2.88,-0.535055,48.0,0.329946,1.177518,74.0,...,-4.645785,3.462669,76.0,-6.949446,4.792778,88.0,-6.165477,3.729443,75.0,-5.390853
22,20221118,02_us_cont_test,sham,2.202290,3.00,1.225633,7.0,-0.869537,1.606031,4.0,...,0.852248,-0.601487,6.0,0.358754,-1.487507,0.0,1.621077,-1.229570,3.0,1.871559
0,20221216,02_us_cont_test,sham,3.332673,2.16,0.463566,64.0,-1.278283,1.687412,97.0,...,-7.121283,2.493121,98.0,-7.451656,1.654931,98.0,-4.509037,1.326684,99.0,-3.925914
26,20221216,09_us_burst_each_intensity,sham,2.010152,1.00,0.898594,12.0,-0.627107,1.599073,35.0,...,-4.084618,4.103973,84.0,-3.985619,5.310014,93.0,-4.187855,6.272437,40.0,-5.060836
0,20220804,05_us_burst_and_sham,sham,0.801040,1.84,3.271857,94.0,-12.326964,3.291230,97.0,...,-2.890858,0.474498,68.0,-2.091701,2.838106,94.0,-10.735775,3.101988,97.0,-11.638241
28,20220819,03_us_burst_sham,sham,2.311712,1.00,0.490820,2.0,-9.914057,-0.177981,14.0,...,-8.448144,0.634381,2.0,-9.180804,3.861800,21.0,-9.067996,4.529794,20.0,-8.350179
29,20220819,02_us_cont_test,sham,2.287576,3.56,-1.536542,0.0,-0.764127,-1.493573,2.0,...,0.522480,-1.513574,5.0,-0.353861,-1.514451,0.0,1.138175,-1.431735,6.0,1.470069
30,20220819,11_after_overdose_sham,sham,2.877168,2.12,2.378619,9.0,-0.603821,2.828229,97.0,...,-0.434608,2.401596,11.0,-0.660225,2.401478,83.0,-0.671505,2.742300,12.0,-0.601446
8,20220914,02_us_cont_test,sham,1.594326,3.92,1.852886,95.0,-7.755265,2.385341,95.0,...,-20.519212,5.246369,94.0,-24.168259,6.782596,94.0,-24.742394,7.682623,93.0,-25.002446


In [8]:
#各パラメータの正規分布をとって見る
stim_parameters = all_sham_datas.columns[3:]
stim_parameters

Index(['abr_z_score', 'abr_latency', 'lfp_z_score_100', 'lfp_latency_100',
       'lfp_ampliture_100', 'lfp_z_score_200', 'lfp_latency_200',
       'lfp_ampliture_200', 'lfp_z_score_300', 'lfp_latency_300',
       'lfp_ampliture_300', 'lfp_z_score_400', 'lfp_latency_400',
       'lfp_ampliture_400', 'lfp_z_score_500', 'lfp_latency_500',
       'lfp_ampliture_500', 'lfp_z_score_600', 'lfp_latency_600',
       'lfp_ampliture_600', 'lfp_z_score_700', 'lfp_latency_700',
       'lfp_ampliture_700', 'lfp_z_score_800', 'lfp_latency_800',
       'lfp_ampliture_800'],
      dtype='object')

In [9]:
#各パラメータに対してF検定を行い正規性の調査p値とF値をpdに保存する
result_df = pd.DataFrame(columns=["w-value","p-value"])
for param in stim_parameters:
    if "z_score" in param:
        res = stats.shapiro(np.abs(all_sham_datas[param]))
    else:
        res = stats.shapiro(all_sham_datas[param])
    result_df.loc[param] = [res.statistic,res.pvalue]
    

In [10]:
result_df

,w-value,p-value
abr_z_score,0.962114,0.671499
abr_latency,0.937463,0.289321
lfp_z_score_100,0.802931,0.002226
lfp_latency_100,0.768771,0.000781
lfp_ampliture_100,0.804517,0.002341
lfp_z_score_200,0.738753,0.000329
lfp_latency_200,0.799457,0.001994
lfp_ampliture_200,0.876772,0.028199
lfp_z_score_300,0.903964,0.079154
lfp_latency_300,0.754655,0.000517


In [15]:
# 30dBのデータとshamのデータのマハラノビス距離を閾値とする
#30dBと20dBの群間分析を行ってマハラノビス距離から境界線を求める
click_30dB=intact_datas.loc[intact_datas["stim_params"].str.extract("(30)\.*\d*dB",expand=False)=="30"]
maharanobis_values =[]
percentage_1_sham = []
percentage_5_sham = []
percentage_10_sham = []
for param in stim_parameters:
    if "z_score" or "ampliture" in param:
        click_30_datas = np.abs(click_30dB[param])
        sham_datas = np.abs(all_sham_datas[param])
    else:
        click_30_datas = click_30dB[param]
        sham_datas = np.abs(all_sham_datas[param])
    click_30dB_mean=np.mean(click_30_datas)
    click_30dB_std=np.std(click_30_datas)
    sham_mean=np.mean(sham_datas)
    sham_std=np.std(sham_datas)

    #マハラノビス距離から境界を算出
    def D1_2(x):
        return (x-click_30dB_mean)**2/click_30dB_std**2
    def D2_2(x):
        return (x-sham_mean)**2/sham_std**2
    for i,x in enumerate(np.arange(sham_mean,70,0.01)):
        score=D1_2(x)-D2_2(x)
        tmp_score=0
        if i!=0:
            if tmp_score > score:
                print(param,x)
                break
        tmp_score=score
    maharanobis_lower =x
    for i,x in enumerate(np.arange(click_30dB_mean,0,-0.01)):
        score=D1_2(x)-D2_2(x)
        tmp_score=0
        if i!=0:
            if tmp_score < score:
                print(param,x)
                break
        tmp_score=score
    maharanobis_upper = x
    maharanobis = min(maharanobis_upper,maharanobis_lower)    
    maharanobis_values.append(maharanobis)
    #片側1%のshamの値を追加
    percentage_1_sham.append(sham_mean+2.33*sham_std)
    percentage_5_sham.append(sham_mean+1.96*sham_std)
    percentage_10_sham.append(sham_mean+1.6449*sham_std)
result_df["maharanobis_value"] = maharanobis_values
result_df["one_side_1%_sham"] = percentage_1_sham
result_df["two_side_5%_sham"] = percentage_5_sham
result_df["one_side_5%_sham"] = percentage_10_sham

abr_z_score 3.23119042857262
abr_z_score 3.2214704128899054
abr_latency 1.1460000000000186
lfp_z_score_100 4.0770680380378295
lfp_z_score_100 4.062712466045369
lfp_latency_100 33.50470588235337
lfp_latency_100 33.5000000000001
lfp_ampliture_100 10.897741325343983
lfp_ampliture_100 10.891638891258737
lfp_z_score_200 4.901279286201328
lfp_z_score_200 4.889852972882068
lfp_latency_200 33.180000000000206
lfp_ampliture_200 12.068033001831969
lfp_ampliture_200 12.063094519978165
lfp_z_score_300 3.920441932203497
lfp_z_score_300 3.916626047594491
lfp_latency_300 38.83647058823497
lfp_latency_300 38.82000000000008
lfp_ampliture_300 12.936561886035776
lfp_ampliture_300 12.930593395004962
lfp_z_score_400 3.7477255058026855
lfp_z_score_400 3.7436266083065703
lfp_latency_400 38.99000000000012
lfp_ampliture_400 13.08738530555866
lfp_ampliture_400 13.078077072097017
lfp_z_score_500 3.5518207729282913
lfp_z_score_500 3.5481380514701186
lfp_latency_500 36.31000000000069
lfp_ampliture_500 11.1359269898

In [16]:
result_df.to_csv("result/shapilo-wilk-test_and_mahanobis-boundary.csv")

In [13]:
result_df.loc["abr_z_score","two_side_5%_sham"]

4.361461566725375